In [1]:
import sympy as sp
x = sp.symbols('x')
import math
from sympy import symbols, sin, cos, exp, ln, log, tan, asin, atan,cot
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter
import random
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
class GenerateFunction:
    def __init__(self, vocab=None, max_depth=2):
        if vocab is None:
            self.vocab = np.array([
                ['add', 6, 2], ['sub', 3, 2], ['mul', 6, 2], ['div', 3, 2],
                ['pow', 3, 2], ['sq', 2, 1], ['sqrt', 2, 1], ['cb', 2, 1],
                ['cbrt', 2, 1], ['exp', 2, 1], ['ln', 2, 1], ['sin', 2, 1],
                ['cos', 2, 1], ['tan', 2, 1], ['asin', 2, 1], ['acos', 2, 1],
                ['atan', 2, 1], ['sinh', 2, 1], ['cosh', 2, 1], ['tanh', 2, 1],
                ['asinh', 2, 1], ['acosh', 2, 1], ['atanh', 2, 1], ['x', 10, 0]
            ])
        else:
            self.vocab = vocab
        self.max_depth = max_depth
        self.x = sp.symbols('x')

    def generate_expression(self, max_depth=None, depth=0, vocab=None):
        if vocab is None:
            vocab = self.vocab
        if max_depth is None:
            max_depth = self.max_depth
            
        if depth >= max_depth:
            return ['x']
        
        weights = vocab[:, 1].astype('float32')
        probs = weights / np.sum(weights)
        N = len(vocab)
        expr = []
        rand_idx = np.random.choice(N, p=probs)
        cur_token = vocab[rand_idx, 0]
        cur_arity = int(vocab[rand_idx, 2])
        expr.append(cur_token)
        
        if cur_arity == 0:
            return expr
        else:
            token_families = [
                ['sin', 'cos', 'tan', 'asin', 'acos', 'atan'],
                ['sinh', 'cosh', 'tanh', 'asinh', 'acosh', 'atanh'],
                ['exp', 'ln'], ['sq', 'sqrt'], ['cb', 'cbrt']
            ]
            token_family = next((f for f in token_families if cur_token in f), None)
            new_vocab = vocab
            if token_family:
                mask = np.isin(vocab[:, 0], token_family, invert=True)
                new_vocab = vocab[mask]
            
            if cur_arity == 1:
                child = self.generate_expression(max_depth, depth+1, new_vocab)
                return expr + child
            elif cur_arity == 2:
                child1 = self.generate_expression(max_depth, depth+1, new_vocab)
                child2 = self.generate_expression(max_depth, depth+1, new_vocab)
                return expr + child1 + child2

    def sequence_to_sympy(self, expr):
        cur_token = expr[0]
        try:
            return float(cur_token)
        except ValueError:
            pass
        
        cur_idx = np.where(self.vocab[:, 0] == cur_token)[0][0]
        cur_arity = int(self.vocab[cur_idx, 2])
        
        if cur_arity == 0:
            return self.x
        elif cur_arity == 1:
            operand = self.sequence_to_sympy(expr[1:])
            return self._handle_unary(cur_token, operand)
        elif cur_arity == 2:
            left_tokens, right_tokens = self._split_expression(expr)
            left_expr = self.sequence_to_sympy(left_tokens)
            right_expr = self.sequence_to_sympy(right_tokens)
            return self._handle_binary(cur_token, left_expr, right_expr)

    def _handle_unary(self, operator, operand):
        operators = {
            'sin': sp.sin, 'cos': sp.cos, 'tan': sp.tan,
            'asin': sp.asin, 'acos': sp.acos, 'atan': sp.atan,
            'sinh': sp.sinh, 'cosh': sp.cosh, 'tanh': sp.tanh,
            'asinh': sp.asinh, 'acosh': sp.acosh, 'atanh': sp.atanh,
            'exp': sp.exp, 'ln': sp.log,
            'sq': lambda x: x**2, 'sqrt': sp.sqrt,
            'cb': lambda x: x**3, 'cbrt': sp.cbrt
        }
        return operators[operator](operand)

    def _handle_binary(self, operator, left, right):
        operators = {
            'add': lambda a,b: a+b, 'sub': lambda a,b: a-b,
            'mul': lambda a,b: a*b, 'div': lambda a,b: a/b,
            'pow': lambda a,b: a**b
        }
        return operators[operator](left, right)

    def _split_expression(self, expr):
        arity_count = 1
        idx_split = 1
        for token in expr[1:]:
            try:
                float(token)
                arity_count -= 1
            except:
                idx = np.where(self.vocab[:, 0] == token)[0][0]
                arity_count += int(self.vocab[idx, 2]) - 1
            if arity_count == 0:
                break
            idx_split += 1
        return expr[1:idx_split], expr[idx_split:]

    def generate_functions(self, num_samples):
        functions = []
        while len(functions) < num_samples:
            try:
                expr = self.generate_expression()
                func = self.sequence_to_sympy(expr)
                functions.append(func)
            except:
                continue
        return functions[:num_samples]

In [3]:
gf = GenerateFunction(max_depth=3)
functions = gf.generate_functions(100)
# Print the first 5 generated functions
for i, func in enumerate(functions[:5]):
    print(f"Function {i+1}: {func}")

Function 1: sin(acosh(x**(1/3)))
Function 2: x
Function 3: x
Function 4: acosh(exp(x))
Function 5: x


# Dataset Preprocessing

In [4]:
# Dataset Preprocessing
class TaylorDataset:
    def __init__(self, order, functions=None):
        self.order = order
        self.x = symbols('x')  # Centralize symbol definition
        self.functions = functions if functions else self.default_functions()
        self.vocab_to_int = None
        self.int_to_vocab = None

    @staticmethod
    def default_functions():
        #x = symbols('x')
        return [
            sin(x), cos(x), exp(x), ln(1 + x), log(1 + x, 10),
            1 / (1 + x), x ** 2 + x + 1, tan(x), asin(x), atan(x),exp(sin(x)),exp(tan(x)), cot(x),
            1/(1+x**2), exp(x)*(1+x), exp(x)*(1-x), 1/(1+x)**2, 1/(1-x)**2, 1/(1-x)**3, 1/(1-x**2), log(3+4*x),
            1/(1+x), 1/(1+x)**2, 1/(1-x), -ln(1-x)
        ]

    def generate(self):
        x = symbols('x')
        data = []
        for func in self.functions:
            try:
                expansion = func.series(x, 0, self.order + 1).removeO()
                # Only add if expansion is different from original function
                if str(func) != str(expansion):
                    data.append({
                        "function": str(func),
                        "expansion": str(expansion),
                        # Optional: include the SymPy objects for debugging
                        "func_obj": func,
                        "expansion_obj": expansion
                    })
            except (NotImplementedError, ValueError, TypeError) as e:
                # Skip functions that can't be expanded
                print(f"Skipping function {func}: {str(e)}")
                continue
            except Exception as e:
                # Catch any other unexpected errors
                print(f"Unexpected error with function {func}: {str(e)}")
                continue
        
        return pd.DataFrame(data).sample(frac=1, random_state=42, ignore_index=True)

    def tokenize(self, df):
        # Tokenize both function and expansion strings.
        # For Taylor expansion tokens, add <SOS> at start and <EOS> at end.
        tokens = []
        for _, row in df.iterrows():
            tokens.extend(word_tokenize(row['function']))
            # add <SOS> and <EOS> for expansions
            exp_tokens = ['<SOS>'] + word_tokenize(row['expansion']) + ['<EOS>']
            tokens.extend(exp_tokens)

        counter = Counter(tokens)
        vocab = sorted(counter, key=counter.get, reverse=True)
        # Ensure special tokens exist:
        for special in ['<SOS>', '<EOS>', '<UNK>']:
            if special not in vocab:
                vocab.append(special)

        self.vocab_to_int = {token: i for i, token in enumerate(vocab, 1)}
        self.int_to_vocab = {i: token for token, i in self.vocab_to_int.items()}

        tokenized_data = {"function_tokens": [], "expansion_tokens": []}

        for _, row in df.iterrows():
            func_tokens = [self.vocab_to_int.get(token, self.vocab_to_int["<UNK>"]) 
                           for token in word_tokenize(row["function"])]
            exp_tokens = (['<SOS>'] + word_tokenize(row["expansion"]) + ['<EOS>'])
            exp_tokens = [self.vocab_to_int.get(token, self.vocab_to_int["<UNK>"]) for token in exp_tokens]
            tokenized_data["function_tokens"].append(func_tokens)
            tokenized_data["expansion_tokens"].append(exp_tokens)
        
        return pd.DataFrame(tokenized_data)
    
    def get_token_dicts(self):
        return self.vocab_to_int, self.int_to_vocab

In [5]:
# Initialize Dataset
order = 4
taylor_dataset = TaylorDataset(order,functions)
df = taylor_dataset.generate()
df

Unexpected error with function sin(acosh(x**(1/3))): Cannot expand sin(acosh(_x**(1/3))) around 0
Skipping function acos(cosh(x)): Could not calculate 5 terms for acos(cosh(_x))
Unexpected error with function acos(log(x**3)): 
Asymptotic expansion of acos around [-oo] is not implemented.
Unexpected error with function cos(log(x)**(1/3)): Cannot expand cos(log(_x)**(1/3)) around 0
Skipping function sqrt(atan(log(x))): 
Unexpected error with function acosh(sin(log(x))): Cannot expand acosh(sin(log(_x))) around 0
Unexpected error with function acos(log(asinh(x))): 
Asymptotic expansion of acos around [-oo] is not implemented.
Unexpected error with function acos(cosh(log(x))): 
Asymptotic expansion of acos around [oo] is not implemented.
Unexpected error with function cos(exp(x**(1/3))): Cannot expand cos(exp(_x**(1/3))) around 0


,function,expansion,func_obj,expansion_obj
0,sin(atanh(x**2)),x**2,sin(atanh(x**2)),x**2
1,tanh(tan(exp(x))),x**4*(-9*tan(1)**5*tanh(tan(1))**4 - 3*tan(1)*...,tanh(tan(exp(x))),x**4*(-9*tan(1)**5*tanh(tan(1))**4 - 3*tan(1)*...
2,exp(2*asinh(x)),x**3 + 2*x**2 + 2*x + 1,exp(2*asinh(x)),x**3 + 2*x**2 + 2*x + 1
3,atanh(sqrt(x)),x**(9/2)/9 + x**(7/2)/7 + x**(5/2)/5 + x**(3/2...,atanh(sqrt(x)),x**(9/2)/9 + x**(7/2)/7 + x**(5/2)/5 + x**(3/2...
4,tan(x),x**3/3 + x,tan(x),x**3/3 + x
5,tan(x)**3,x**3,tan(x)**3,x**3
6,tanh(x),-x**3/3 + x,tanh(x),-x**3/3 + x
7,cos(x),x**4/24 - x**2/2 + 1,cos(x),x**4/24 - x**2/2 + 1
8,sinh(exp(tan(x))),x**4*(2*sinh(1)/3 + 5*cosh(1)/8) + x**3*(sinh(...,sinh(exp(tan(x))),x**4*(2*sinh(1)/3 + 5*cosh(1)/8) + x**3*(sinh(...
9,sinh(x),x**3/6 + x,sinh(x),x**3/6 + x


In [6]:
tokenized_df = taylor_dataset.tokenize(df)
tokenized_df

,function_tokens,expansion_tokens
0,"[57, 1, 66, 1, 95, 2, 2]","[25, 95, 26]"
1,"[37, 1, 23, 1, 52, 1, 24, 2, 2, 2]","[25, 58, 1, 141, 1, 5, 2, 67, 1, 23, 1, 5, 2, ..."
2,"[52, 1, 39, 1, 24, 2, 2]","[25, 87, 3, 157, 3, 158, 3, 5, 26]"
3,"[66, 1, 40, 1, 24, 2, 2]","[25, 34, 1, 77, 2, 99, 3, 34, 1, 88, 2, 159, 3..."
4,"[23, 1, 24, 2]","[25, 100, 3, 24, 26]"
5,"[23, 1, 24, 2, 22]","[25, 87, 26]"
6,"[37, 1, 24, 2]","[25, 101, 3, 24, 26]"
7,"[62, 1, 24, 2]","[25, 120, 6, 79, 3, 5, 26]"
8,"[55, 1, 52, 1, 23, 1, 24, 2, 2, 2]","[25, 58, 1, 161, 1, 5, 2, 41, 3, 162, 1, 5, 2,..."
9,"[55, 1, 24, 2]","[25, 121, 3, 24, 26]"


In [28]:
tokenized_df = tokenized_df[
    (tokenized_df['function_tokens'].apply(len) <= 10000) &
    (tokenized_df['expansion_tokens'].apply(len) <= 10000)
]

# PyTorch Dataset and Collate Function

In [7]:
class TrainDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        data = self.dataset.iloc[idx]
        # function_tokens are encoder input; expansion_tokens are decoder target.
        function_tensor = torch.tensor(data['function_tokens'], dtype=torch.long)
        expansion_tensor = torch.tensor(data['expansion_tokens'], dtype=torch.long)
        return function_tensor, expansion_tensor

def collate_fn(batch):
    # batch: list of (src, trg) pairs.
    src_seqs, trg_seqs = zip(*batch)
    src_padded = nn.utils.rnn.pad_sequence(src_seqs, batch_first=True, padding_value=0)
    trg_padded = nn.utils.rnn.pad_sequence(trg_seqs, batch_first=True, padding_value=0)
    return src_padded, trg_padded

# Positional Encoding

In [24]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=10000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)  # [max_len, d_model]
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)  # [max_len, 1]
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)  # even indices
        pe[:, 1::2] = torch.cos(position * div_term)  # odd indices
        pe = pe.unsqueeze(0)  # [1, max_len, d_model]
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        # x: [batch, seq_len, d_model]
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

# Transformer Seq2Seq Model

In [25]:
class TransformerModel(nn.Module):
    def __init__(self, input_dim, output_dim, embed_size, nhead, num_layers, hidden_size, dropout=0.1):
        """
        input_dim: size of source vocabulary (including padding idx=0)
        output_dim: size of target vocabulary (including padding idx=0)
        embed_size: embedding dimension (d_model)
        nhead: number of attention heads
        num_layers: number of encoder and decoder layers
        hidden_size: dimension of the feedforward network
        """
        super().__init__()
        self.embed_size = embed_size
        
        self.src_embedding = nn.Embedding(input_dim, embed_size, padding_idx=0)
        self.tgt_embedding = nn.Embedding(output_dim, embed_size, padding_idx=0)
        
        self.pos_encoder = PositionalEncoding(embed_size, dropout)
        self.pos_decoder = PositionalEncoding(embed_size, dropout)
        
        self.transformer = nn.Transformer(d_model=embed_size, nhead=nhead, 
                                          num_encoder_layers=num_layers, 
                                          num_decoder_layers=num_layers, 
                                          dim_feedforward=hidden_size,
                                          dropout=dropout)
        self.fc_out = nn.Linear(embed_size, output_dim)
    
    def make_tgt_mask(self, tgt):
        # Generate a square subsequent mask for the target sequence.
        tgt_seq_len = tgt.shape[1]
        tgt_mask = torch.triu(torch.ones(tgt_seq_len, tgt_seq_len) == 1, diagonal=1)
        tgt_mask = tgt_mask.float().masked_fill(tgt_mask, float('-inf'))
        return tgt_mask

    def forward(self, src, tgt):
        """
        src: [batch, src_len]
        tgt: [batch, tgt_len]
        """
        src_emb = self.src_embedding(src) * math.sqrt(self.embed_size)  # [batch, src_len, embed_size]
        src_emb = self.pos_encoder(src_emb)  # [batch, src_len, embed_size]
        tgt_emb = self.tgt_embedding(tgt) * math.sqrt(self.embed_size)  # [batch, tgt_len, embed_size]
        tgt_emb = self.pos_decoder(tgt_emb)  # [batch, tgt_len, embed_size]
        
        # Transformer expects input shape: [seq_len, batch, embed_size]
        src_emb = src_emb.transpose(0, 1)
        tgt_emb = tgt_emb.transpose(0, 1)
        
        tgt_mask = self.make_tgt_mask(tgt).to(src.device)
        
        output = self.transformer(src_emb, tgt_emb, tgt_mask=tgt_mask)
        # output: [tgt_len, batch, embed_size]
        output = output.transpose(0, 1)  # [batch, tgt_len, embed_size]
        output = self.fc_out(output)     # [batch, tgt_len, output_dim]
        # Applying log softmax for NLLLoss
        return nn.functional.log_softmax(output, dim=-1)

# Training Class for Transformer

In [26]:
class Train:
    def __init__(self, epoch, batch_size, input_dim, output_dim, embed_size, nhead, num_layers, hidden_size, dropout=0.1):
        self.epoch = epoch
        self.batch_size = batch_size
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        self.model = TransformerModel(input_dim, output_dim, embed_size, nhead, num_layers, hidden_size, dropout).to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        self.criterion = nn.NLLLoss(ignore_index=0)
    
    def run(self, dataloader):
        for epoch in range(self.epoch):
            self.model.train()
            epoch_loss = 0
            for src, tgt in dataloader:
                src, tgt = src.to(self.device), tgt.to(self.device)
                self.optimizer.zero_grad()
                # During training, we feed the full target sequence
                output = self.model(src, tgt[:, :-1])
                # output: [batch, tgt_len-1, output_dim]
                # Compare against target tokens shifted by one (i.e. from position 1 onward)
                output = output.reshape(-1, output.shape[-1])
                target = tgt[:, 1:].reshape(-1)
                loss = self.criterion(output, target)
                loss.backward()
                self.optimizer.step()
                epoch_loss += loss.item()
            if (epoch + 1) % 100 == 0 or epoch == 0:
                print(f'Epoch {epoch+1} - Loss: {epoch_loss:.4f}')
    
    def get_model(self):
        return self.model.to("cpu")

# Initialize Dataset and Training Setup

In [29]:
vocab_to_int, int_to_vocab = taylor_dataset.get_token_dicts()
dataset = TrainDataset(tokenized_df)
train_loader = DataLoader(dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)

# Hyperparameters for Transformer
epoch = 500
batch_size = 1
embed_size = 32
nhead = 4
num_layers = 2
hidden_size = 64
input_dim = len(vocab_to_int) + 1  # +1 for padding index 0
output_dim = len(vocab_to_int) + 1

trainer = Train(epoch, batch_size, input_dim, output_dim, embed_size, nhead, num_layers, hidden_size, dropout=0.1)
trainer.run(train_loader)

Epoch 1 - Loss: 225.9542
Epoch 100 - Loss: 16.9551
Epoch 200 - Loss: 9.9038
Epoch 300 - Loss: 9.6007
Epoch 400 - Loss: 7.8239
Epoch 500 - Loss: 7.4067


# Prediction Function for Transformer

In [30]:
def predict_sample(model, src_tensor, vocab_to_int, int_to_vocab, max_len=30):
    """
    Predicts the output sequence for a given input sequence (src_tensor) using the trained transformer model.
    Uses greedy decoding.
    """
    model.eval()
    device = next(model.parameters()).device
    src_tensor = src_tensor.unsqueeze(0).to(device)  # [1, src_len]
    
    # Encode source
    with torch.no_grad():
        src_emb = model.src_embedding(src_tensor) * math.sqrt(model.embed_size)
        src_emb = model.pos_encoder(src_emb).transpose(0,1)  # [src_len, 1, embed_size]
        memory = model.transformer.encoder(src_emb)
    
    # Start with <SOS>
    sos_token = vocab_to_int["<SOS>"]
    eos_token = vocab_to_int["<EOS>"]
    tgt_indices = [sos_token]
    
    for _ in range(max_len):
        tgt_tensor = torch.tensor(tgt_indices, dtype=torch.long, device=device).unsqueeze(0)  # [1, t]
        tgt_emb = model.tgt_embedding(tgt_tensor) * math.sqrt(model.embed_size)
        tgt_emb = model.pos_decoder(tgt_emb).transpose(0,1)  # [t, 1, embed_size]
        tgt_mask = model.make_tgt_mask(tgt_tensor).to(device)
        
        with torch.no_grad():
            output = model.transformer.decoder(tgt_emb, memory, tgt_mask=tgt_mask)
        output = output.transpose(0,1)  # [1, t, embed_size]
        output = model.fc_out(output)    # [1, t, output_dim]
        output = nn.functional.log_softmax(output, dim=-1)
        next_token = output[0, -1].argmax().item()
        if next_token == eos_token:
            break
        tgt_indices.append(next_token)
    
    predicted_tokens = [int_to_vocab[idx] for idx in tgt_indices[1:]]
    return predicted_tokens

In [31]:
trained_model = trainer.get_model()
trained_model.eval()

TransformerModel(
  (src_embedding): Embedding(268, 32, padding_idx=0)
  (tgt_embedding): Embedding(268, 32, padding_idx=0)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (pos_decoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-1): 2 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
          )
          (linear1): Linear(in_features=32, out_features=64, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=64, out_features=32, bias=True)
          (norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2):

# Prediction Example

In [34]:
# Pick a sample from the dataset (for example, first sample)
sample_input, sample_target = dataset[2]
predicted_expansion = predict_sample(trained_model, sample_input, vocab_to_int, int_to_vocab, max_len=30)
print("Function Tokens (input):", [int_to_vocab[token] for token in sample_input.tolist()])
print("Predicted Expansion:", " ".join(predicted_expansion))

Function Tokens (input): ['exp', '(', '2*asinh', '(', 'x', ')', ')']
Predicted Expansion: x**3 + 2*x**2 + 2*x + 1
